In [ ]:
## Airbnb Bookings Data Analysis
#Exploring listing and bookings data to uncover factors that drive cancellations, revenue potential, and guest behavior.

## 1. Objective
#Initial EDA and vizualisations to identify the drivers of cancellations, revenue and guest behavior

## 2. Data Overview
#This data is publicly available on kaggle.com and contains

## 3. Cleaning & Preparation
#Missing data, formatting, value sanity checks.

## 4. Key Questions Explored
# What listing features correlate with high occupancy?
# Does cancellation policy impact pricing or reviews?
# How does location affect guest behavior?

## 5. Visual Insights
#Charts, grouped metrics, comparisons.

## 6. Observations & Takeaways
#Summary of business-relevant insights.
